In [1]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
xm = load_model('transmitter', device=device) # reconstruction
model = load_model('text300M', device=device) # text embedding
diffusion = diffusion_from_config(load_config('diffusion')) # latent diffusion

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
batch_size = 4
guidance_scale = 15.0
prompt = "a pyramid"

latent = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh
import open3d as o3d

t = decode_latent_mesh(xm, latent[0]).tri_mesh()
with open('intermediate_mesh.ply', 'wb') as f:
        t.write_ply(f)
mesh = o3d.io.read_triangle_mesh('intermediate_mesh.ply')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Bin size was too small in the coarse rasterization phase. This caused an overflow, meaning output may be incomplete. To solve, try increasing max_faces_per_bin / max_points_per_bin, decreasing bin_size, or setting bin_size to 0 to use the naive rasterization.Bin size was too small in the coarse rasterization phase. This caused an overflow, meaning output may be incomplete. To solve, try increasing max_faces_per_bin / max_points_per_bin, decreasing bin_size, or setting bin_size to 0 to use the naive rasterization.Bin size was too small in the coarse rasterization phase. This caused an overflow, meaning output may be incomplete. To solve, try increasing max_faces_per_bin / max_points_per_bin, decreasing bin_size, or setting bin_size to 0 to use the naive rasterization.Bin size was too small in the coarse rasterization pha

In [ ]:
# clean the mesh
import pymeshfix

v, f = pymeshfix.clean_from_arrays(mesh.vertices, mesh.triangles)
fixed_mesh = o3d.geometry.TriangleMesh()
fixed_mesh.vertices = o3d.utility.Vector3dVector(v)
fixed_mesh.triangles = o3d.utility.Vector3iVector(f)
o3d.visualization.draw_geometries([fixed_mesh])

In [ ]:
# simplify the mesh
fixed_mesh = fixed_mesh.simplify_quadric_decimation(2048)
o3d.visualization.draw_geometries([fixed_mesh])

In [ ]:
# check if the mesh is watertight
watertight = fixed_mesh.is_watertight()
print("\nWatertight: ", watertight)

if watertight:
    # calculate the mesh volume
    v = o3d.geometry.TriangleMesh.get_volume(fixed_mesh)
    print("\nVolume: ", v)

    # scale the mesh to a specified volume
    target_volume = 5
    scale_factor = (target_volume / v) ** (1/3)
    fixed_mesh.scale(scale_factor, center=fixed_mesh.get_center())
    o3d.visualization.draw_geometries([fixed_mesh])
    print("\nVolume: ", o3d.geometry.TriangleMesh.get_volume(fixed_mesh))

In [ ]:
# sample the mesh to a point cloud
alternate_point_cloud = fixed_mesh.sample_points_poisson_disk(2048)
o3d.visualization.draw_geometries([alternate_point_cloud])